In [1]:
import requests
import pandas as pd
import numpy as np
import os
import time
import datetime
import json

In [2]:
csv_list = [file for file in os.listdir() if file.endswith("boxoffice.csv")]

In [3]:
for i, data in enumerate(csv_list):
    if i == 0:
        df = pd.read_csv(data)
    else:
        small_df = pd.read_csv(data)
        df = pd.concat([df, small_df]).reset_index(drop=True)

In [4]:
df.tail()

,순위,영화명,개봉일,매출액,매출액 점유율,관객수,스크린수,상영횟수,대표국적,국적,배급사
23429,2217,BISFF HONOR 2,NaN,0,0.0%,0,1,2,한국,한국,NaN
23430,2217,BISFF HOT HOT,NaN,0,0.0%,0,1,2,기타,기타,NaN
23431,2217,[인디포럼2016] 기획전03 김지현특별전 02,NaN,0,0.0%,0,1,1,한국,한국,NaN
23432,2217,영향 아래 있는 여자,NaN,0,0.0%,0,1,3,NaN,NaN,NaN
23433,합계,NaN,NaN,"820,067,267,748",NaN,"102,278,002","78,970","3,027,941",NaN,NaN,NaN


## 관객 수 10000명 이상

In [5]:
df["관객수"] = df["관객수"].str.replace(",","")

In [6]:
df["관객수"] = df["관객수"].astype(int)

In [7]:
movie_df = df[df["관객수"] > 10000].reset_index(drop=True)

In [8]:
movie_df = movie_df.drop(movie_df.index[[2569]])

In [9]:
movie_df = movie_df.drop_duplicates("영화명").reset_index(drop=True)

In [10]:
movieNm = movie_df["영화명"]

### movie_df to csv

In [11]:
movie_df.to_csv("movie.csv",encoding="utf-8")

## get_movie_data

In [12]:
def get_movie_data(movieNm):
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"
    params = {"key":"74ab5419e95d3e66cd760d6f498dc423", "movieNm":movieNm}
    r = requests.get(url, params=params)
    return r.json()

In [13]:
def make_movie_df(movieNm):
    movie_df = pd.DataFrame(columns = ["movieCd", "movieNm", "director", "openDt", "typeNm", "repNationNm", "repGenreNm", "companyNm"])
    for i in list(movieNm):
        try:
            for data in get_movie_data(i)['movieListResult']['movieList']:
                if len(data["directors"]) >= 2:
                    director = data["directors"][0]["peopleNm"]
                elif len(data["directors"]) == 1:
                    director = data["directors"][0]["peopleNm"]
                if len(data["companys"]) >= 2:
                    companyNm = data["companys"][0]["companyNm"]
                elif len(data["companys"]) == 1:
                    companyNm = data["companys"][0]["companyNm"] 
                movie_df.loc[len(movie_df)] = [
                    data["movieCd"],
                    data["movieNm"],
                    director,
                    data["openDt"],
                    data["typeNm"],
                    data["repNationNm"],
                    data["repGenreNm"],
                    companyNm
                ]
        except:
            print(i)
    return movie_df

In [14]:
movie_info_df = make_movie_df(movieNm)

황후화
도쿄!
그들 각자의 영화관
시네마 천국


In [16]:
movie_info_df = movie_info_df[~movie_info_df["movieNm"].str.contains("시네마정동")].reset_index(drop=True)

In [17]:
movie_info_df = movie_info_df[movie_info_df["repNationNm"] != "기타"].reset_index(drop=True)

In [18]:
movie_info_df = movie_info_df.drop_duplicates("movieNm").reset_index(drop=True)

In [19]:
movie_info_df.to_csv("movie_info.csv",encoding="utf-8")

In [48]:
movie_info_df.tail(20)

,Unnamed: 0,movieCd,movieNm,director,openDt,typeNm,repNationNm,repGenreNm,companyNm
3000,3000,20158905,버스 657,스콧 만,20160114,장편,미국,액션,미디어 아시아 필름즈
3001,3001,20040703,"조제, 호랑이 그리고 물고기들",이누도 잇신,20041029,장편,일본,드라마,아스믹에이스엔터테인먼트
3002,3002,20163081,미라클 프롬 헤븐,패트리시아 리건,20160525,장편,미국,드라마,아스믹에이스엔터테인먼트
3003,3003,20158982,트릭,이창열,20160713,장편,한국,스릴러,엘씨오픽쳐스
3004,3004,20154201,일렉트릭 슬라이드,트리스탄 패터슨,20150820,장편,미국,범죄,엘씨오픽쳐스
3005,3005,20164747,리틀 폭스,갈수영,20160623,장편,중국,애니메이션,엘씨오픽쳐스
3006,3006,19590006,벤허,윌리엄 와일러,19720907,장편,미국,드라마,메트로-골드윈-마이어
3007,3007,20152541,무수단,구모,20160303,장편,한국,미스터리,골든타이드픽처스(주)
3008,3008,20154902,자객 섭은낭,허우 샤오시엔,20160204,장편,대만,사극,골든타이드픽처스(주)
3009,3009,20156083,트윈스터즈,사만다 푸터먼,20160303,장편,미국,다큐멘터리,골든타이드픽처스(주)


In [58]:
def get_movie_detail(movieCd):
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"
    params = {"key":"3805474b27f1763b1aaffa6702ea06e6", "movieCd":movieCd}
    r = requests.get(url, params=params)
    return r.json()

In [14]:
movie_info_df = pd.read_csv("movie_info.csv")

In [15]:
movieCd = movieCd.astype(str)

In [57]:
movieCd = movie_info_df["movieCd"]

In [44]:
def make_movie_detail_df(movie_info_df):
    movie_detail_df = pd.DataFrame(columns=["movieCd", 
                                            "movieNm", 
                                            "showTm", 
                                            "watchGradeNm", 
                                            "actor_1",
                                            "actor_2",
                                            "actor_3",
                                            "companyNm"
                                            ])
    for i in list(movieCd):
        try:
            data = get_movie_detail(i)['movieInfoResult']['movieInfo']
            actor_list = []
            if len(data["audits"]) >= 2:
                watchGradeNm = data["audits"][0]["watchGradeNm"]
            elif len(data["audits"]) == 1:
                watchGradeNm = data["audits"][0]["watchGradeNm"]
            if len(data["companys"]) >= 2:
                companyNm = data["companys"][0]["companyNm"]
            elif len(data["companys"]) == 1:
                companyNm = data["companys"][0]["companyNm"] 
            if len(data["actors"]) >= 3:
                actor_list = [
                    data["actors"][0]["peopleNm"],
                    data["actors"][1]["peopleNm"],
                    data["actors"][2]["peopleNm"]
                ]
            else:
                for i in range(len(data["actors"])):
                    actor_list.append(data["actors"][i]["peopleNm"])
                for i in range(3-len(data["actors"])):
                    actor_list.append("")
            movie_detail_df.loc[len(movie_detail_df)] = [
                data["movieCd"],
                data["movieNm"],
                data["showTm"],
                watchGradeNm,
                *actor_list,
                companyNm
            ]
        except:
             print(i)
    return movie_detail_df

In [59]:
movie_detail_df = make_movie_detail_df(movieCd)

20158905
20040703
20163081
20158982
20154201
20164747
19590006
20152541
20154902
20156083
20156080
20161598
20164387
20159965
20155174
20153182
20020227
20157441
20163441
20162101


In [52]:
movieCd = movie_info_df["movieCd"][3000::]

In [61]:
len(movie_detail_df)

3000

In [63]:
len(movie_detail1_df)

20

In [64]:
movie_detail = pd.concat([movie_detail_df, movie_detail1_df]).reset_index(drop=True)

In [66]:
movie_detail.to_csv("movie_detail.csv",encoding="utf-8")

In [86]:
boxoffice_df = movie_info_df.merge(movie_detail, left_on="movieNm", right_on="movieNm")[[
        "movieCd_x",
        "movieNm",
        "director",
        "openDt",
        "typeNm",
        "repNationNm",
        "repGenreNm",
        "showTm",
        "watchGradeNm",
        "actor_1",
        "actor_2",
        "actor_3",
        "companyNm_y"
    ]]

In [88]:
boxoffice_df = boxoffice_df.rename(columns={"movieCd_x":"movieCd", "companyNm_y":"companyNm"})

In [90]:
boxoffice_df.to_csv("boxoffice.csv",encoding="utf-8")